In [19]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [21]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,64.81,29,59,5.01,broken clouds
1,1,Kaseda,JP,31.4167,130.3167,57.83,54,84,16.22,broken clouds
2,2,Sao Filipe,CV,14.8961,-24.4956,71.06,78,4,8.79,clear sky
3,3,Pangnirtung,CA,66.1451,-65.7125,-0.40,71,0,3.44,clear sky
4,4,Pout,SN,14.7739,-17.0603,67.98,94,40,12.66,scattered clouds


In [22]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_prompt_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_prompt_temp = float(input('What is the maximum temperature you would like for your trip? '))                    
raining_prompt = input("Do you want it to be raining? (yes/no) ")
snowing_prompt = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [23]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_prompt_temp) & \
                                       (city_data_df["Max Temp"] >= min_prompt_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Sao Filipe,CV,14.8961,-24.4956,71.06,78,4,8.79,clear sky
5,5,Salinas,US,36.6777,-121.6555,73.99,58,0,19.57,clear sky
7,7,Atuona,PF,-9.8000,-139.0333,79.00,78,19,15.43,light rain
8,8,Muisne,EC,0.6000,-80.0333,76.84,86,96,7.99,overcast clouds
9,9,Tura,IN,25.5198,90.2201,73.76,93,70,10.22,broken clouds


In [24]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Sao Filipe,CV,71.06,clear sky,14.8961,-24.4956,
5,Salinas,US,73.99,clear sky,36.6777,-121.6555,
7,Atuona,PF,79.00,light rain,-9.8000,-139.0333,
8,Muisne,EC,76.84,overcast clouds,0.6000,-80.0333,
9,Tura,IN,73.76,broken clouds,25.5198,90.2201,
10,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,
18,Eyl,SO,82.27,overcast clouds,7.9803,49.8164,
20,Namibe,AO,71.42,broken clouds,-15.1961,12.1522,
24,Concepcion Del Oro,MX,80.98,overcast clouds,24.6333,-101.4167,
26,Rikitea,PF,77.56,clear sky,-23.1203,-134.9692,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))